In [1]:
#1.install and import dependencies
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
#2.Using MP Holistic to get the keypoints
mp_holistic = mp.solutions.holistic #Holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

In [3]:
#functions declarations
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # 468*3 + 33*4 + 21*3 + 21*3 
    return np.concatenate([pose, face, lh, rh])

In [4]:
#New!!!
##for testing collecting new set of data and identify whether there is problem with the test data##
#Setup folders to store the keypoints
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('NEW_DATA')

#Actions that we try to detect
actions = np.array(['Hi','Terima Kasih',"Saya Sayang Awak" ,"Awak", "Makan"])

#Thirty videos worth of data
no_sequences = 30

#Each of the video will have 30 frames in length
sequence_length = 30

#Folder start
start_folder = 30

for action in actions:
    #dirmax  = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            #make directory NEW_DATA/hello/0
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            #if there's exception just ignore it and keep going
            pass


In [1]:
#New!!!
##for testing collecting new set of data and identify whether there is problem with the test data##
# 5. Collecting keypoints values for training and testing 
cap = cv2.VideoCapture(0)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in actions:
        print(f"Get ready for action: {action}")
        
        # Capture a frame to display the preparation message
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break
        
        # Display preparation message
        cv2.putText(frame, f"Get ready for action: {action}", (50, 200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.imshow("Sign Language Recognition", frame)
        cv2.waitKey(5000)  # Pause for 5 seconds
        
        # Loop through sequences (videos)
        for sequence in range(no_sequences):
            # Loop through video length (sequence length)
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame. Exiting...")
                    break
                
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Display messages
                if frame_num == 0:  # Display starting message
                    cv2.putText(image, "Starting Collection", (120, 200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.imshow("Sign Language Recognition", image)
                    cv2.waitKey(2000)  # Wait 2 seconds at the start of each sequence
                else:
                    cv2.putText(image, f"Collecting frames for {action} Video Number {sequence}", (15, 12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")
                os.makedirs(os.path.dirname(npy_path), exist_ok=True)
                np.save(npy_path, keypoints)
                
                # Show the frame
                cv2.imshow("Sign Language Recognition", image)

                # Break gracefully when q key is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [7]:
#New!!!
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical

label_map = {action: idx for idx, action in enumerate(actions)}  # Map actions to integers
# Prepare sequences and labels
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# Convert to NumPy arrays
x = np.array(sequences)  # Shape: (num_sequences, sequence_length, num_keypoints)
y = to_categorical(labels)  # Convert labels to one-hot encoding

In [ ]:
#New!!!
#split data into 70% training data and 30% testing data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

print("Training data shape:", x_train.shape)
print("Testing data shape:", x_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)

In [9]:
#New!!!
#Model Creation
#LSTM
model = Sequential([
    LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, x.shape[2])),
    LSTM(128, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(actions), activation='softmax')  # Output layer matches the number of actions
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 549,509 (2.10 MB)

 Trainable params: 549,509 (2.10 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#New!!!
#Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=380,  # Adjust epochs based on your data
    batch_size=32  # Adjust batch size based on your hardware
)

In [ ]:
#New!!!
#Evaluate the Model
#Use the test data to calculate the confusion matrix and accuracy score

# Predict on the test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true_classes = np.argmax(y_test, axis=1)  # Convert one-hot labels to class labels

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Accuracy Score
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print("Accuracy Score:", accuracy)

In [ ]:
#New!!!
#Visualize the Confusion Matrix
#Use a heatmap to visualize the confusion matrix and give better insight into model performance.

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=actions, yticklabels=actions)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
#New!!!
#9.Save the model 
model.save("recognition_model_LSTM_v3.keras")

In [11]:
#New!!!
model.load_weights("recognition_model_LSTM_v3.keras")

In [14]:
#New!!!
#testing the model
#new variables
sequence = []
sentence = []
predictions = []
threshold = 0.75

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame  = cap.read()
        
        #Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(image, results)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #prediction logic 
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        #get the latest 30 keypoints
        sequence = sequence[-30:]
        
        #make prediction every 30 frame
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print("Predicted Action: " + actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            #Viz logic: to check whether the last prediction is the same as the current one
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 1:
                sentence = sentence[-1:]
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        #Show to screen
        cv2.imshow("Sign Language Recognition", image)
         
        #Break gracefully when q key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

[[[220 216 211]
  [220 216 211]
  [222 218 213]
  ...
  [ 34  28  25]
  [ 31  28  23]
  [ 29  27  22]]

 [[222 218 213]
  [222 218 213]
  [223 219 214]
  ...
  [ 32  26  23]
  [ 28  24  19]
  [ 28  25  20]]

 [[222 219 212]
  [222 219 212]
  [222 219 212]
  ...
  [ 31  24  21]
  [ 24  18  14]
  [ 26  20  15]]

 ...

 [[186 166 159]
  [184 163 156]
  [185 163 155]
  ...
  [ 74  55  44]
  [ 69  49  37]
  [118  96  84]]

 [[183 164 163]
  [183 164 162]
  [183 163 159]
  ...
  [ 74  51  41]
  [ 74  49  42]
  [ 79  53  47]]

 [[189 171 173]
  [186 167 168]
  [182 164 162]
  ...
  [ 69  44  35]
  [ 70  43  37]
  [ 69  41  37]]] <class 'mediapipe.python.solution_base.SolutionOutputs'>
[[[224 222 212]
  [222 220 210]
  [224 222 212]
  ...
  [ 41  32  27]
  [ 37  32  25]
  [ 32  29  21]]

 [[223 221 211]
  [223 221 211]
  [224 222 212]
  ...
  [ 38  29  24]
  [ 37  32  25]
  [ 32  29  21]]

 [[223 221 211]
  [224 222 212]
  [223 222 212]
  ...
  [ 38  30  25]
  [ 33  28  22]
  [ 29  26  18]]

 

In [ ]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame  = cap.read()
        
        #Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(image, results)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #Show to screen
        cv2.imshow("OpenCV Feed", image)
        
        #Break gracefully when q key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


In [ ]:
#3. Extract the keypoint values
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)
#print(pose)

In [ ]:
result_test  = extract_keypoints(results)
#np.save('0',result_test)
#np.load('0.npy')

In [ ]:
# 4.Setup folders to store the keypoints
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA')#the parent folder's path will starts with MP_DATA

#Actions that we try to detect
actions = np.array(['Hi','Terima Kasih', 'Apa Khabar',"Saya Sayang Awak" ,
                    "Awak", "Minta Maaf", "Makan", "Minum",
                    "Salah","Betul", "Saya", "Tolong"])

#Thirty videos worth of data
no_sequences = 30

#Each of the video will have 30 frames in length
sequence_length = 30

#Folder start
start_folder = 30

In [ ]:
#new actions
new_actions = ["Salah","Betul", "Saya", "Tolong"]
for action in new_actions:
    #dirmax  = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            #make directory MP_DATA/hello/0
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            #if there's exception just ignore it and keep going
            pass

In [ ]:
# 5.Collecting keypoints values for training and testing 
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #new loop 
    #loop through actions
    for action in new_actions:
        #loop through sequences aka videos
        for sequence in range(no_sequences):
            #loop through video length aka sequence length
            for frame_num in range(sequence_length):
                #Read feed
                ret, frame  = cap.read()
                
                #Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                #draw landmarks
                draw_styled_landmarks(image, results)
                
                #new apply wait logic to give breaks
                if frame_num == 0:
                    cv2.putText(image, "Starting Collection", (120, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, "Collecting frames for {} Video Number {}".format(action, sequence), (15, 12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 1, cv2.LINE_AA)
                    #Show to screen
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, "Collecting frames for {} Video Number {}".format(action, sequence), (15, 12),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0, 255), 1, cv2.LINE_AA) 
                    
                cv2.imshow("Sign Language Recognition", image)
                #new export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                #Break gracefully when q key is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
#6. Preprocess data and create labels and features
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label: num for num, label in enumerate(actions)}
label_map

In [ ]:
# 30 sequences = 30 videos recorded , labels = y labels
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH,action))).astype(int):
        #to store each of the video
        window = []
        #to append each of the frame together to form a complete video
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(labels).shape

In [ ]:
x = np.array(sequences)
x.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.05)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
#7. Build and train LSTM Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Input
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
#Build Neural Network with CNN + LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout

In [ ]:
#Build Neural Network with RCNN + LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, GRU, LSTM, Dense, Dropout, Bidirectional

In [ ]:
#to store the training logs, can view the progress using cmd: tensorboard --logdir=.
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
#LSTM
model = Sequential()
#have to return the sequences so that the next layer can use them
# Specify the input using Input layer
model.add(Input(shape=(30, 1662)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#softmax means the max value would be 1 and 1 would be allocated to each of the different actions as probability
model.add(Dense(actions.shape[0],activation='softmax'))
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
#CNN + LSTM
# Build the model
model = Sequential()

# Input Layer
model.add(Input(shape=(30, 1662)))

# 1D Convolutional Layers for Feature Extraction
model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# LSTM Layers for Temporal Dependencies
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Fully Connected Dense Layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output Layer with Softmax Activation for Multi-class Classification
model.add(Dense(actions.shape[0], activation='softmax'))

# Compile the Model
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
# Build the RCNN + LSTM model
model = Sequential()

# Input Layer
model.add(Input(shape=(30, 1662)))

# RCNN (Recurrent Convolutional Layers)
model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
model.add(GRU(64, return_sequences=True))  # Adding a recurrent layer to the convolutional output
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model.add(GRU(128, return_sequences=True))  # Recurrent connection in the second convolutional layer
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

# LSTM Layers for Temporal Dependencies
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Fully Connected Dense Layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output Layer with Softmax Activation for Multi-class Classification
model.add(Dense(actions.shape[0], activation='softmax'))

# Compile the Model
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
#train the model **this will takes time
model.fit(x_train, y_train, epochs=1500, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
#8.Make predictions
res = model.predict(x_test)
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[3])]

In [ ]:
#9.Save the model 
model.save("recognition_model_RCNN_LSTM_v1.keras")

In [ ]:
#delete the model
del model

In [ ]:
model.load_weights("recognition_model_RCNN_LSTM_v1.keras")

In [ ]:
#10. Evaluation using confusion matrix and accuracy
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
#11.Test in real time
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
#function to draw the probability bars on each of the actions
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90 +num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

In [ ]:
#plot the output figure
plt.figure(figsize=(9,9))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
#testing the model
#new variables
sequence = []
sentence = []
predictions = []
threshold = 0.75

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame  = cap.read()
        
        #Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(image, results)
        #print(results)
        
        #draw landmarks
        draw_styled_landmarks(image, results)
        
        #prediction logic 
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        #get the latest 30 keypoints
        sequence = sequence[-30:]
        
        #make prediction every 30 frame
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print("Predicted Action: " + actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            #Viz logic: to check whether the last prediction is the same as the current one
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 1:
                sentence = sentence[-1:]
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        #Show to screen
        cv2.imshow("Sign Language Recognition", image)
         
        #Break gracefully when q key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break



In [ ]:
cap.release()
cv2.destroyAllWindows()